# Combinando conjuntos de datos: Concat y Append

Algunos de los estudios de datos más interesantes provienen de la combinación de diferentes fuentes de datos.
Estas operaciones pueden implicar cualquier cosa, desde una concatenación muy sencilla de dos conjuntos de datos diferentes hasta uniones y fusiones más complicadas al estilo de una base de datos que manejan correctamente cualquier superposición entre los conjuntos de datos.
``Series`` y ``DataFrame`` se construyen con este tipo de operación en mente, y Pandas incluye funciones y métodos que hacen que este tipo de procesamiento de datos sea rápido y sencillo.

Aquí veremos la concatenación simple de ``Series`` y ``DataFrame`` con la función ``pd.concat``; Más adelante profundizaremos en fusiones y uniones en memoria más sofisticadas implementadas en Pandas.

Comenzamos con las importaciones estándar:

In [1]:
import pandas as pd
import numpy as np

Por conveniencia, definiremos esta función que crea un ``DataFrame`` de una forma particular que será útil a continuación:

In [2]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


Además, crearemos una clase rápida que nos permita mostrar múltiples ``DataFrame`` uno al lado del otro. El código hace uso del método especial ``_repr_html_``, que IPython utiliza para implementar su visualización enriquecida de objetos:

In [3]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)

    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)


El uso de esto quedará más claro a medida que continuemos nuestra discusión en la siguiente sección.

## Recordar: Concatenación de matrices NumPy

La concatenación de objetos ``Series`` y ``DataFrame`` es muy similar a la concatenación de matrices Numpy, que se puede realizar mediante la función ``np.concatenate`` como se explica en la parte de Numpy.
Recuerde que con él puede combinar el contenido de dos o más matrices en una sola matriz:

In [4]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

El primer argumento es una lista o tupla de matrices para concatenar.
Además, se necesita una palabra clave ``eje`` que le permite especificar el eje a lo largo del cual se concatenará el resultado:

In [16]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

## Concatenación simple con ``pd.concat``

Pandas tiene una función, ``pd.concat()``, que tiene una sintaxis similar a ``np.concatenate`` pero contiene una serie de opciones que discutiremos en un momento:

```python
# Signature in Pandas v0.18
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```

``pd.concat()`` se puede usar para una concatenación simple de objetos ``Series`` o ``DataFrame``, al igual que ``np.concatenate()`` se puede usar para concatenaciones simples de matrices:

In [36]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
print(ser1)
ser2 = pd.Series(['D', 'E', 'F'], index=[1, 2, 3])
print(ser2)
pd.concat([ser1, ser2], axis = 0).reset_index(drop=True)

1    A
2    B
3    C
dtype: object
1    D
2    E
3    F
dtype: object


0    A
1    B
2    C
3    D
4    E
5    F
dtype: object

También funciona para concatenar objetos de dimensiones superiores, como ``DataFrames``:

In [42]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [1, 2])

display('df1','df2','pd.concat([df1, df2], axis=1)')

df1
    A   B
1  A1  B1
2  A2  B2

df2
    A   B
1  A1  B1
2  A2  B2

pd.concat([df1, df2], axis=1)
    A   B   A   B
1  A1  B1  A1  B1
2  A2  B2  A2  B2

De forma predeterminada, la concatenación se realiza por filas dentro del ``DataFrame`` (es decir, ``axis=0``).
Al igual que ``np.concatenate``, ``pd.concat`` permite especificar un eje a lo largo del cual se realizará la concatenación.
Considere el siguiente ejemplo:

In [40]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis=0)")

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis=0)
     A    B    C    D
0   A0   B0  NaN  NaN
1   A1   B1  NaN  NaN
0  NaN  NaN   C0   D0
1  NaN  NaN   C1   D1

### Índices duplicados

Una diferencia importante entre ``np.concatenate`` y ``pd.concat`` es que la concatenación de Pandas *conserva los índices*, ¡incluso si el resultado tendrá índices duplicados!
Considere este sencillo ejemplo:

In [45]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!
display('x', 'y', 'pd.concat([x, y], axis = 0)') # reset_index()

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


Observe los índices repetidos en el resultado.
Si bien esto es válido dentro de los ``DataFrames``, el resultado suele ser indeseable.
``pd.concat()`` nos brinda algunas formas de manejarlo.

#### Detectar las repeticiones como un error

Si desea simplemente verificar que los índices en el resultado de ``pd.concat()`` no se superpongan, puede especificar el indicador ``verify_integrity``.
Con esto establecido en Verdadero, la concatenación generará una excepción si hay índices duplicados.
Aquí hay un ejemplo, donde para mayor claridad capturaremos e imprimiremos el mensaje de error:

In [46]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Index([0, 1], dtype='int64')


#### Ignorando el índice

A veces el índice en sí no importa y usted preferiría simplemente ignorarlo.
Esta opción se puede especificar usando el indicador ``ignore_index``.
Con esto establecido en verdadero, la concatenación creará un nuevo índice entero para la ``Serie`` resultante:

In [50]:
#display('x', 'y', 'pd.concat([x, y], ignore_index=True)')
display('x', 'y', 'pd.concat([x, y]).reset_index(drop=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


#### Agregar claves MultiIndex

Otra opción es utilizar la opción ``keys`` para especificar una etiqueta para las fuentes de datos; el resultado será una serie indexada jerárquicamente que contiene los datos:

In [51]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['x', 'y'])
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

### Concatenación con uniones

En los ejemplos simples que acabamos de ver, principalmente estábamos concatenando ``DataFrames`` con nombres de columnas compartidos.
En la práctica, los datos de diferentes fuentes pueden tener diferentes conjuntos de nombres de columnas y ``pd.concat`` ofrece varias opciones en este caso.
Considere la concatenación de los siguientes dos ``DataFrames``, que tienen algunas (¡pero no todas!) columnas en común:

In [57]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
# display(df5)
# display(df6)
display('df5', 'df6', 'pd.concat([df5, df6])')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

De forma predeterminada, las entradas para las que no hay datos disponibles se completan con valores NA.
Para cambiar esto, podemos especificar una de varias opciones para los parámetros ``join`` y ``join_axes`` de la función concatenar.
Por defecto, la unión es una unión de las columnas de entrada (``join='outer'``), pero podemos cambiar esto a una intersección de las columnas usando ``join='inner'``:

In [60]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner' ,axis=1)")
# join=inner es interseccion, en este caso lo hace por
# filas, por ende como los indices no coinciden me da
# vacio

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner' ,axis=1)
Empty DataFrame
Columns: [A, B, C, B, C, D]
Index: []

In [62]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner' ,axis=0)")
# en este caso lo hace por columnas, coinciden B y C

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner' ,axis=0)
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4

La combinación de opciones de la función ``pd.concat`` permite una amplia gama de comportamientos posibles al unir dos conjuntos de datos; téngalos en cuenta al utilizar estas herramientas para sus propios datos.

In [64]:
pd.__version__ #append esta deprecado

'2.1.1'